# Notebook utilising the combined_all.nc dataset

Will try and flopt some of the individual magnetic field discharges

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import sys
import os
import glob
import re
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.core.fitters as fts

In [2]:
importlib.reload(pd)
importlib.reload(plt)
importlib.reload(mpl)

<module 'matplotlib' from '/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/matplotlib/__init__.py'>

In [2]:
path_to_datasets = '/home/jleland/Data/external/magnum/'
# path_to_analysed_datasets = 'analysed_2'
path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'analysed_3_downsampled'
os.chdir(path_to_datasets)

In [3]:
all_ds = xr.open_dataset('all_combined_ds.nc')
all_ds

<xarray.Dataset>
Dimensions:                 (direction: 2, probe: 4, shot_number: 440, time: 500, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 3 4 5 6 7 ... 478 479 480 481
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * probe                   (probe) object 'B' 'L' 'R' 'S'
  * direction               (direction) object 'up' 'down'
Dimensions without coordinates: time
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64

In [ ]:
fig, ax = plt.subplots(3, sharex=True)
all_ds.where(np.logical_and(all_ds.shot_b_field < 1.21, all_ds.shot_b_field > 1.19), drop=True).max('ts_radial_pos')['shot_b_field'].plot(marker='+', ax=ax[0])
all_ds.where(np.logical_and(all_ds.shot_b_field < 1.21, all_ds.shot_b_field > 1.19), drop=True).max('ts_radial_pos')['ts_temp_max'].plot(marker='+', ax=ax[1])
all_ds.where(np.logical_and(all_ds.shot_b_field < 1.21, all_ds.shot_b_field > 1.19), drop=True).max('ts_radial_pos')['adc_index'].plot(marker='+', ax=ax[2])
plt.show()

In [4]:
axial_scan_indices = set([148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189])

In [5]:
# ds_1_2T = all_ds.where(np.logical_and(all_ds.shot_b_field < 1.21, all_ds.shot_b_field > 1.19), drop=True).max('ts_radial_pos')

# Hydrogen shots @ tilt = 10 that isn't the axial scan
ds_1_2T = all_ds.where(np.logical_and(all_ds.shot_tilt.round() == 10.0, all_ds.shot_hydrogen_gf > 0.1), drop=True) #.swap_dims({'shot_number': 'shot_b_field'})
ds_1_2T = ds_1_2T.sel(shot_number=list(set(ds_1_2T.shot_number.values) - axial_scan_indices)).sortby('shot_number')
ds_1_2T = ds_1_2T.max('ts_radial_pos')

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [ ]:
fig, ax = plt.subplots(2, 2, sharex=True)
ds_1_2T['ts_temp_max'].plot(x='shot_number', marker='+', linestyle='none', ax=ax[0][0])
ds_1_2T['ts_dens_max'].plot(x='shot_number', marker='+', linestyle='none', ax=ax[1][0])
ds_1_2T['shot_hydrogen_gf'].plot(x='shot_number', marker='+', linestyle='none', ax=ax[0][1], label='H')
ds_1_2T['shot_helium_gf'].plot(x='shot_number', marker='+', linestyle='none', ax=ax[0][1], label='He')
ax[0][1].legend()
ds_1_2T['shot_source_current'].plot(x='shot_number', linestyle='none', marker='+', ax=ax[1][1])

plt.show()

In [ ]:
fig, ax = plt.subplots(2, sharex=True)
ds_1_2T.set_coords('shot_b_field')['ts_temp_max'].plot(x='shot_b_field', marker='+', linestyle='none', ax=ax[0])
ds_1_2T.set_coords('shot_b_field')['ts_dens_max'].plot(x='shot_b_field', marker='+', linestyle='none', ax=ax[1])

plt.show()

In [ ]:
ds_1_2T

In [ ]:
# This changes the colormap for all of matplotlib - not strictly necessary to run
colourmap = plt.get_cmap('nipy_spectral')
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[colourmap(k) for k in np.linspace(0, 1, 12)])

In [ ]:
iv_fig, iv_ax = plt.subplots(2, 2, sharex=True)
for i, probe in enumerate(ds_1_2T['probe'].values):
    iv_ax[i%2][i//2].set_title(probe)
    
    ds = ds_1_2T.sel(probe=probe).mean('direction')
    ds.set_coords('voltage')['current'].plot.line(ax=iv_ax[i%2][i//2], x='voltage', hue='shot_number')
    iv_ax[i%2][i//2].set_ylabel('I (A)')
    iv_ax[i%2][i//2].set_xlabel(r'$V_P$ (V)')
    
# plt.tight_layout()
plt.show()

In [ ]:
fit_df = pd.DataFrame(columns=['shot',
                               'probe', 
                               'B',
                               'ts_temp',
                               'ts_dens',
                               'temp',
                               'd_temp',
                               'isat', 
                               'd_isat',
                               'a',
                               'd_a',
                               'v_f',
                               'd_v_f'
                              ])
fit_df

In [6]:
mandatory_labels = [
    'shot',
    'probe', 
    'B',
    'ts_temp',
    'ts_dens',
    'fit_success_fl',
]
fit_param_labels = [
    'temp',
    'd_temp',
    'isat', 
    'd_isat',
    'a',
    'd_a',
    'v_f',
    'd_v_f',                              
]
all_labels = mandatory_labels + fit_param_labels
fit_df = pd.DataFrame(columns=all_labels)

probes = ['S', 'L', 'B', 'R']

for shot in ds_1_2T.shot_number.values:
    shot_ds = ds_1_2T.sel(shot_number=shot).mean('direction')
    for probe in probes:
        probe_shot_ds = shot_ds.sel(probe=probe)
        probe_shot_ds = probe_shot_ds.where(np.isfinite(probe_shot_ds['voltage']), drop=True)
        
        if len(probe_shot_ds.time) == 0:
            continue
#         probe_shot_ds = probe_shot_ds
        
#         print(probe_shot_ds)
        shot_iv = iv.IVData(probe_shot_ds['voltage'].values,
                            -probe_shot_ds['current'].values,
                            probe_shot_ds['shot_time'].values,
                            estimate_error_fl=True)
        
        fit_params = {}
                
        try:
            shot_fit = shot_iv.multi_fit(sat_region=-60)
            fit_params = {
                'fit_success_fl': True,
                'temp': shot_fit.get_temp(),
                'd_temp': shot_fit.get_temp_err(),
                'isat': shot_fit.get_isat(), 
                'd_isat': shot_fit.get_isat_err(),
                'a': shot_fit.get_sheath_exp(), 
                'd_a': shot_fit.get_sheath_exp_err(),
                'v_f': shot_fit.get_floating_pot(),
                'd_v_f': shot_fit.get_floating_pot_err()
            }
        except RuntimeError as e:
            print(f'WARNING: Failed on shot {shot} with probe {probe}')
            fit_params = {label: np.NaN for label in fit_param_labels}
            fit_params['fit_success_fl'] = False
            
        
        fit_df = fit_df.append({
            'shot': shot,
            'probe': probe, 
            'B': np.around(probe_shot_ds['shot_b_field'].mean().values, decimals=1),
            'ts_temp': probe_shot_ds['ts_temperature'].mean().values,
            'ts_dens': probe_shot_ds['ts_density'].mean().values,
            **fit_params
        }, ignore_index=True)
        

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Projects/flopter/flopter/core/fitters.py:216: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + (a * np.float_power(np.absolute(V), [0.75])))
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


...

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops

In [7]:
fit_df['ts_temp'] = fit_df['ts_temp'].astype(np.float64)
fit_df['ts_dens'] = fit_df['ts_dens'].astype(np.float64)
fit_df

,shot,probe,B,ts_temp,ts_dens,fit_success_fl,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,22,S,0.5,0.328979,2.925608e+19,True,0.938039,2.860422e-02,0.027699,0.000089,2.173877e-02,0.000483,-7.131685,0.015998
1,22,L,0.5,0.328979,2.925608e+19,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24,S,0.5,0.319234,2.442321e+19,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24,L,0.5,0.319234,2.442321e+19,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,25,S,0.5,0.330994,2.637601e+19,True,682151.979440,1.323535e+08,59.601764,11556.011968,4.217276e-20,0.084520,52.902013,18.905666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,453,L,0.8,NaN,NaN,True,3.145238,9.341682e-02,0.007193,0.000061,1.816498e-02,0.001129,-25.154440,0.039320
134,454,S,0.8,NaN,NaN,True,3.022893,1.643838e-02,0.084727,0.000099,5.886546e-02,0.000224,-27.972588,0.008388
135,454,L,0.8,NaN,NaN,True,1.805120,8.936862e-02,0.009771,0.000072,1.421596e-02,0.000685,-11.718765,0.048979
136,455,S,0.8,3.125829,1.331822e+19,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
save_filename = 'concatted_fits.csv'

In [22]:
fit_df.to_csv(save_filename)

In [7]:
fit_df = pd.read_csv(save_filename)
fit_df

,Unnamed: 0,shot,probe,B,ts_temp,ts_dens,fit_success_fl,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,0,22,S,0.5,0.328979,2.925608e+19,True,0.938039,2.860422e-02,0.027699,0.000089,2.173877e-02,0.000483,-7.131685,0.015998
1,1,22,L,0.5,0.328979,2.925608e+19,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,24,S,0.5,0.319234,2.442321e+19,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,24,L,0.5,0.319234,2.442321e+19,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,25,S,0.5,0.330994,2.637601e+19,True,682151.979440,1.323535e+08,59.601764,11556.011968,4.217276e-20,0.084520,52.902013,18.905666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,133,453,L,0.8,NaN,NaN,True,3.145238,9.341682e-02,0.007193,0.000061,1.816498e-02,0.001129,-25.154440,0.039320
134,134,454,S,0.8,NaN,NaN,True,3.022893,1.643838e-02,0.084727,0.000099,5.886546e-02,0.000224,-27.972588,0.008388
135,135,454,L,0.8,NaN,NaN,True,1.805120,8.936862e-02,0.009771,0.000072,1.421596e-02,0.000685,-11.718765,0.048979
136,136,455,S,0.8,3.125829,1.331822e+19,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# s_df = fit_df.where(fit_df.probe == 'S').dropna()
s_df = fit_df.where(fit_df.probe == 'S').dropna(how='all')#.drop(columns='probe')
s_df

,Unnamed: 0,shot,probe,B,ts_temp,ts_dens,fit_success_fl,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,0.0,22.0,S,0.5,0.328979,2.925608e+19,1.0,0.938039,2.860422e-02,0.027699,0.000089,2.173877e-02,0.000483,-7.131685,0.015998
2,2.0,24.0,S,0.5,0.319234,2.442321e+19,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,25.0,S,0.5,0.330994,2.637601e+19,1.0,682151.979440,1.323535e+08,59.601764,11556.011968,4.217276e-20,0.084520,52.902013,18.905666
6,6.0,26.0,S,0.5,0.356140,3.130804e+19,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.0,27.0,S,0.5,0.340792,2.783332e+19,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,126.0,339.0,S,1.2,4.023214,1.771685e+19,1.0,5.426847,1.960196e-02,0.153816,0.000302,3.277288e-02,0.000492,-40.557930,0.006210
130,130.0,436.0,S,1.5,5.379086,1.118215e+20,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,132.0,453.0,S,0.8,NaN,NaN,1.0,3.972853,3.046450e-02,0.080740,0.000215,5.925751e-02,0.000488,-40.460050,0.012679
134,134.0,454.0,S,0.8,NaN,NaN,1.0,3.022893,1.643838e-02,0.084727,0.000099,5.886546e-02,0.000224,-27.972588,0.008388


In [8]:
l_df = fit_df.where(fit_df.probe == 'L').dropna(how='all')#.drop(columns='probe')
l_df

,Unnamed: 0,shot,probe,B,ts_temp,ts_dens,fit_success_fl,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
1,1.0,22.0,L,0.5,0.328979,2.925608e+19,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,24.0,L,0.5,0.319234,2.442321e+19,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,25.0,L,0.5,0.330994,2.637601e+19,1.0,10.054462,1.151776,0.000186,0.000047,5.014608,0.901783,-30.908799,0.382557
7,7.0,26.0,L,0.5,0.356140,3.130804e+19,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,27.0,L,0.5,0.340792,2.783332e+19,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,127.0,339.0,L,1.2,4.023214,1.771685e+19,1.0,2.859447,0.021696,0.049403,0.000085,0.019290,0.000228,-24.922060,0.011717
131,131.0,436.0,L,1.5,5.379086,1.118215e+20,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,133.0,453.0,L,0.8,NaN,NaN,1.0,3.145238,0.093417,0.007193,0.000061,0.018165,0.001129,-25.154440,0.039320
135,135.0,454.0,L,0.8,NaN,NaN,1.0,1.805120,0.089369,0.009771,0.000072,0.014216,0.000685,-11.718765,0.048979


In [11]:
s_temp_fraction = s_df['ts_temp'] / s_df['temp']
d_s_temp_fraction = s_temp_fraction * np.sqrt((0.05)**2 + (s_df['d_temp'] / s_df['temp'])**2)

mag_grouped = s_df.groupby('B', as_index=False)
mag_avg_df = mag_grouped.mean()
# mag_avg_df = mag_avg_df.assign(ts_temp=l_df[['B', 'ts_temp']].dropna().groupby('B').mean())
mag_std_df = mag_grouped.std()

s_avg_frac = mag_avg_df['ts_temp'] / mag_avg_df['temp']
s_B = mag_avg_df['B']
d_s_avg_frac = s_avg_frac * np.sqrt((0.05)**2 + (mag_std_df['d_temp'] / mag_std_df['temp'])**2)

plt.figure()
plt.errorbar(s_df['B'], s_temp_fraction, yerr=d_s_temp_fraction, fmt='x')
plt.errorbar(s_B, s_avg_frac, yerr=d_s_avg_frac)
plt.axhline(y=1.0, color='k', linestyle='dashed', linewidth=0.5)
plt.show()

In [12]:
l_temp_fraction = l_df['ts_temp'] / l_df['temp']
d_l_temp_fraction = l_temp_fraction * np.sqrt((0.05)**2 + (l_df['d_temp'] / l_df['temp'])**2)

mag_grouped = l_df.groupby('B', as_index=False)
mag_avg_df = mag_grouped.mean()
# mag_avg_df = mag_avg_df.assign(ts_temp=l_df[['B', 'ts_temp']].dropna().groupby('B').mean())
mag_std_df = mag_grouped.std()

l_avg_frac = mag_avg_df['ts_temp'] / mag_avg_df['temp']
l_B = mag_avg_df['B']
d_l_avg_frac = l_avg_frac * np.sqrt((0.05)**2 + (mag_std_df['d_temp'] / mag_std_df['temp'])**2)

plt.figure()
plt.errorbar(l_df['B'], l_temp_fraction, yerr=d_l_temp_fraction, fmt='x')
plt.errorbar(l_B, l_avg_frac, yerr=d_l_avg_frac)
plt.axhline(y=1.0, color='k', linestyle='dashed', linewidth=0.5)
plt.show()

In [12]:
plt.figure()
plt.errorbar(s_df['B'], s_temp_fraction, yerr=d_s_temp_fraction, fmt='x', label='S')
plt.errorbar(l_df['B'], l_temp_fraction, yerr=d_l_temp_fraction, fmt='x', label='L')
plt.errorbar(s_B, s_avg_frac, yerr=d_s_avg_frac, label='S', color='darkblue')
plt.errorbar(l_B, l_avg_frac, yerr=d_l_avg_frac, label='L', color='orange')
plt.axhline(y=1.0, color='k', linestyle='dashed', linewidth=0.5)
plt.ylabel(r'$T_{TS}$ / $T_{P}$')
plt.xlabel(r'$B$')
plt.show()
plt.title(r'Temperature measurements grouped by Magnetic field at $\theta = 10^{\circ}$')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
singlemag_df = fit_df.where(fit_df.B == 0.8).dropna(how='all')
singlemag_df

,Unnamed: 0,shot,probe,B,ts_temp,ts_dens,fit_success_fl,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
16,16.0,47.0,S,0.8,1.197945,2.438797e+20,1.0,2.247919,0.023888,0.264937,0.000551,0.018241,0.000218,-8.274396,0.011964
17,17.0,47.0,L,0.8,1.197945,2.438797e+20,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,18.0,58.0,B,0.8,1.191589,2.138598e+20,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,19.0,58.0,R,0.8,1.191589,2.138598e+20,1.0,0.988479,0.020276,0.004556,0.000009,0.032453,0.000480,-9.142898,0.012453
20,20.0,86.0,B,0.8,3.165614,1.388232e+19,1.0,6.546069,0.035057,0.080968,0.000143,0.039363,0.000332,-12.331105,0.016724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,133.0,453.0,L,0.8,NaN,NaN,1.0,3.145238,0.093417,0.007193,0.000061,0.018165,0.001129,-25.154440,0.039320
134,134.0,454.0,S,0.8,NaN,NaN,1.0,3.022893,0.016438,0.084727,0.000099,0.058865,0.000224,-27.972588,0.008388
135,135.0,454.0,L,0.8,NaN,NaN,1.0,1.805120,0.089369,0.009771,0.000072,0.014216,0.000685,-11.718765,0.048979
136,136.0,455.0,S,0.8,3.125829,1.331822e+19,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
fig, ax = plt.subplots()

colors = np.where(singlemag_df["probe"]=='S','r','-')
colors[singlemag_df["probe"]=='L'] = 'b'
colors[singlemag_df["probe"]=='B'] = 'y'
colors[singlemag_df["probe"]=='R'] = 'g'
# print(colors)


singlemag_df.plot.scatter(ax=ax, x='ts_temp', y='temp', marker='x', c=colors)
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'k', zorder=0, linewidth=0.75, linestyle='dotted')
ax.set_aspect('equal')
ax.set_xlim(lims)
ax.set_ylim(lims)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.5059630099801777, 8.053814626765078)

In [14]:
fig, ax = plt.subplots()

colors = np.where(fit_df["probe"]=='S','C0','-')
colors[fit_df["probe"]=='L'] = 'C1'
colors[fit_df["probe"]=='B'] = 'k'
colors[fit_df["probe"]=='R'] = 'r'
print(colors)


fit_df.plot.scatter(ax=ax, x='ts_temp', y='temp', marker='x', c=colors)
lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]
print(lims)

# now plot both limits against eachother
ax.plot(lims, lims, 'k', zorder=0, linewidth=0.75, linestyle='dotted')
ax.set_aspect('equal')
ax.set_xlim(0,10)
ax.set_ylim(0,10)

['C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1'
 'C0' 'C1' 'C0' 'C1' 'k' 'r' 'k' 'r' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0'
 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0'
 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0'
 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0'
 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0'
 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0'
 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0'
 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0'
 'C1' 'k' 'r' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1' 'C0' 'C1']
[-34106.92327598158, 716259.5525742897]


(0, 10)

In [8]:
fig, axes = plt.subplots(1, 3)

for i, b_field in enumerate([0.5, 0.8, 1.2]):
    ax = axes[i]
    df = fit_df.where(fit_df.B == b_field).dropna(how='all')

    colors = np.where(df["probe"]=='S','C0','-')
    colors[df["probe"]=='L'] = 'C1'
    colors[df["probe"]=='B'] = 'r'
    colors[df["probe"]=='R'] = 'g'

    labels, index = np.unique(colors, return_inverse=True)
    print(labels, index)
    labels = ['S', 'L', 'B', 'R']
    
    sc = ax.scatter(x='ts_temp', y='temp', data=df, marker='x', c=index)
    lims = [
        0,   # min of both axes
        10,  # max of both axes
    ]
    lims_linspace = np.linspace(0, 10, 100)
    
    sl_fitter = fts.StraightLineFitter()
    
    df = df.dropna()
    fit_data = sl_fitter.fit(df['ts_temp'], df['temp'], sigma=df['d_temp'])

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k', zorder=0, linewidth=0.75, linestyle='dotted')
    ax.plot(lims_linspace, fit_data.fit_function(lims_linspace), color='red', linestyle='dashed', label='fit')
    ax.set_aspect('equal')
    ax.set_xlim(0,10)
    ax.set_ylim(0,10)
    ax.set_title(f'B = {b_field}T')
    ax.legend(sc.legend_elements()[0], labels)
#     if i == 0:
#         ax.legend({'C0': 'S', 'C1': 'L', 'r': 'B', 'g': 'R'})

['C0' 'C1'] [0 1 0 1 0 1 0 1 0 1 0 1 0 1]
['C0' 'C1' 'g' 'r'] [0 1 3 2 3 2 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
['C0' 'C1' 'g' 'r'] [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 3 2]


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/matplotlib/colors.py:933: UserWarning: Warning: converting a masked element to nan.
  dtype = np.min_scalar_type(value)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/ma/core.py:713: UserWarning: Warning: converting a masked element to nan.
  data = np.array(a, copy=False, subok=subok)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/matplotlib/ticker.py:589: UserWarning: Warning: converting a masked element to nan.
  s = self.format % xp


In [21]:
for shot in fit_df.where(fit_df['fit_success_fl']).dropna()['shot'].values[0:5]:
    # Get the relevant rows from each database
    iv_row = fit_df.where(fit_df.shot == shot).dropna(how='all')
    ds_info = ds_1_2T.sel(shot_number=shot).mean('direction')
#     print(ds_info)
    
    fig, ax = plt.subplots(1, 2, sharex=True)
    for i, probe in enumerate(ds_info['probe'].values):
        ax[i//2].set_title(probe)

        ds = ds_info.sel(probe=probe)
        if len(ds.where(np.isfinite(ds.voltage), drop=True)['voltage']) == 0:
            continue
        ds.set_coords('voltage')['current'].plot.line(ax=ax[i//2], x='voltage')
        ax[i//2].set_ylabel('I (A)')
        ax[i//2].set_xlabel(r'$V_P$ (V)')

    # plt.tight_layout()
    plt.show()
    
#     ax.plot(iv_row[''])

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


## Finding what happened to the 1.5T shots

In [17]:
dsβ = all_ds.where(np.logical_and(all_ds.shot_b_field < 1.51, all_ds.shot_b_field > 1.49), drop=True).max('ts_radial_pos')
dsβ = dsβ.set_coords('shot_source_current')

fig, ax = plt.subplots(2, 2, sharex=True)
dsβ['ts_temp_max'].plot(x='shot_source_current', marker='+', linestyle='none', ax=ax[0][0])
dsβ['ts_dens_max'].plot(x='shot_source_current', marker='+', linestyle='none', ax=ax[1][0])
dsβ['shot_hydrogen_gf'].plot(x='shot_source_current', marker='+', linestyle='none', ax=ax[0][1], label='H')
dsβ['shot_helium_gf'].plot(x='shot_source_current', marker='+', linestyle='none', ax=ax[0][1], label='He')
ax[0][1].legend()
dsβ['shot_source_current'].plot(x='shot_source_current', linestyle='none', marker='+', ax=ax[1][1])

plt.show()

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plt.close('all')

In [19]:
fig, ax = plt.subplots(2, 2, sharex=True)
dsβ['ts_temp_max'].plot(x='shot_number', marker='+', linestyle='none', ax=ax[0][0])
dsβ['ts_dens_max'].plot(x='shot_number', marker='+', linestyle='none', ax=ax[1][0])
dsβ['shot_hydrogen_gf'].plot(x='shot_number', marker='+', linestyle='none', ax=ax[0][1], label='H')
dsβ['shot_helium_gf'].plot(x='shot_number', marker='+', linestyle='none', ax=ax[0][1], label='He')
ax[0][1].legend()
dsβ['shot_source_current'].plot(x='shot_number', linestyle='none', marker='+', ax=ax[1][1])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
iv_fig, iv_ax = plt.subplots(2, 2, sharex=True)
for i, probe in enumerate(dsβ['probe'].values):
    iv_ax[i%2][i//2].set_title(probe)
    
    ds = dsβ.sel(probe=probe).mean('direction')
    ds.set_coords('voltage')['current'].plot.line(ax=iv_ax[i%2][i//2], x='voltage', hue='shot_number')
    iv_ax[i%2][i//2].set_ylabel('I (A)')
    iv_ax[i%2][i//2].set_xlabel(r'$V_P$ (V)')
    
# plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


## Looking at errors in analysis trimming

In [4]:
all_ds

<xarray.Dataset>
Dimensions:                 (direction: 2, probe: 4, shot_number: 440, time: 500, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 3 4 5 6 7 ... 478 479 480 481
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * probe                   (probe) object 'B' 'L' 'R' 'S'
  * direction               (direction) object 'up' 'down'
Dimensions without coordinates: time
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64

In [5]:
s_all_ds = all_ds.sel(probe='S')
s_all_ds = s_all_ds.where(np.isfinite(s_all_ds.voltage), drop=True).mean('direction').max('ts_radial_pos').min('time')['shot_time'].plot.line(marker='+')
plt.show()

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [6]:
fig, ax = plt.subplots(2)
all_ds.mean('direction').max('ts_radial_pos').max('sweep')['start_time'].plot.line(ax=ax[0], marker='+', linestyle='none', hue='probe')
all_ds.mean('direction').max('ts_radial_pos').min('sweep')['start_time'].plot.line(ax=ax[1], marker='+', linestyle='none', hue='probe')
plt.show()

ValueError: Dataset does not contain the dimensions: ['sweep']